In [ ]:
# https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#eq-9


In [4]:
import numpy as np
from tqdm import tqdm

# open('/Users/georgychernousov/Downloads/example.txt','r')
with open('reviews.txt','r') as f:
    reviews = f.readlines()
    
# __init__
tokens = ' '.join(reviews).split()[:500000]
vocab = list(set(tokens))
batch_size = 100
hidden_size = 100
learning_rate = 0.01
window = 1
X, y = [], []
w_input = (np.random.rand(len(vocab),hidden_size))#  * 0.001
w_output = (np.random.rand(hidden_size,len(vocab)))# * 0.001

def encode_one_hot(x, vocab_size):
    one_hots = []
    for i in x:
        zeros = np.zeros([vocab_size, 1])
        zeros[i] = 1
        one_hots.append(zeros)
    return one_hots

def softmax(x):
    return np.divide(np.exp(x), np.sum(np.exp(x), axis=0, keepdims=True) + 0.001)

fallten = lambda x: np.array([item for sublist in x for item in sublist])

for word_i in tqdm(range(len(tokens))):
    target_word = vocab.index(tokens[word_i])
    y_indexes = [*list(range(max(0,word_i-window), word_i)), 
        *list(range(word_i+1, min(len(tokens),word_i+window+1)))
        ]
    y.append(y_indexes)
    X.append([target_word] * len(y_indexes))
    
    if word_i % batch_size == 0 and word_i != 0:
        one_hots = encode_one_hot(fallten(X), len(vocab))
        # forward prop
        preds = []
        l_1s = []
        for x in one_hots:
            l_1 = np.dot(x.reshape(1, -1), w_input)
            l_1s.append(l_1)
            # softmax
            l_2 = np.dot(l_1, w_output)
            y_pred = softmax(l_2)
            preds.append(y_pred)
        
        # back prop
        pred_index = 0
        preds_to_sum = []
        y_trues_to_sum = []
        for y_true in y:
            y_trues = encode_one_hot(np.array(y_true), len(vocab))
            y_trues = [y_true.reshape(1, -1) for y_true in y_trues]
            y_trues_to_sum.append(y_trues)
            preds_to_sum.append(preds[pred_index:len(y_true)+pred_index])
            pred_index += len(y_true)
        sum_errors = []
        for i in range(len(y)):
            sum_error = sum(preds_to_sum[i]) - sum(y_trues_to_sum[i])
            sum_errors.append(sum_error)
        grad_w_inputs = []
        grad_w_outputs = []
        for n, sum_error in enumerate(sum_errors):
            grad_w_inputs.append(
                np.dot(encode_one_hot(X[n], len(vocab))[0], 
                       np.dot(w_output, 
                              sum_error.reshape(-1, 1)).T
                      )
            )
            grad_w_outputs.append(
                np.dot(
                    l_1.reshape(-1, 1),
                    sum_error
                )
            )
        # update
        for i in range(len(y)):
            w_input -= learning_rate * grad_w_inputs[i]
            w_output -= learning_rate * grad_w_outputs[i]
        X, y = [], []
        break

  0%|          | 100/500000 [00:05<6:58:48, 19.89it/s]
